# Colab 연습

In [2]:
!cat /etc/issue.net #지금나의시스템확인

Ubuntu 18.04.5 LTS


In [3]:
!head /proc/cpuinfo #Xeon79 CPU구나...

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 63
model name	: Intel(R) Xeon(R) CPU @ 2.30GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2299.998
cache size	: 46080 KB
physical id	: 0


In [4]:
!head -n 3 /proc/meminfo #메모리용량확인

MemTotal:       13333596 kB
MemFree:         1772336 kB
MemAvailable:    3428236 kB


In [5]:
!nvidia-smi #이따 GPU로 바꾸면 나옴

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [6]:
!pwd #지금 위치

/content


In [7]:
!ls -l #sample_data폴더가 만들어져있다

total 4
drwxr-xr-x 1 root root 4096 Feb  4 15:26 sample_data


In [8]:
!ls -l sample_data #폴더내용을보자~

total 55504
-rwxr-xr-x 1 root root     1697 Jan  1  2000 anscombe.json
-rw-r--r-- 1 root root   301141 Feb  4 15:26 california_housing_test.csv
-rw-r--r-- 1 root root  1706430 Feb  4 15:26 california_housing_train.csv
-rw-r--r-- 1 root root 18289443 Feb  4 15:26 mnist_test.csv
-rw-r--r-- 1 root root 36523880 Feb  4 15:26 mnist_train_small.csv
-rwxr-xr-x 1 root root      930 Jan  1  2000 README.md


In [9]:
!cat sample_data/README.md #파일내용보기

This directory includes a few sample datasets to get you started.

*   `california_housing_data*.csv` is California housing data from the 1990 US
    Census; more information is available at:
    https://developers.google.com/machine-learning/crash-course/california-housing-data-description

*   `mnist_*.csv` is a small sample of the
    [MNIST database](https://en.wikipedia.org/wiki/MNIST_database), which is
    described at: http://yann.lecun.com/exdb/mnist/

*   `anscombe.json` contains a copy of
    [Anscombe's quartet](https://en.wikipedia.org/wiki/Anscombe%27s_quartet); it
    was originally described in

    Anscombe, F. J. (1973). 'Graphs in Statistical Analysis'. American
    Statistician. 27 (1): 17-21. JSTOR 2682899.

    and our copy was prepared by the
    [vega_datasets library](https://github.com/altair-viz/vega_datasets/blob/4f67bdaad10f45e3549984e17e1b3088c731503d/vega_datasets/_data/anscombe.json).


In [10]:
!python --version #현pc는 3.8.3인데 여기는 3.6.9구버전(똑같이f-string을 쓸수있다) 3.6ver이상부터~

Python 3.6.9


In [12]:
import tensorflow as tf
tf.__version__  #2.4.1 초반1대의버전보다는 훨 좋다;

'2.4.1'

In [13]:
from tensorflow import keras
keras.__version__ #2.4.0

'2.4.0'

In [14]:
print('한글')

한글


### MNIST 손글씨 분류

#### 1. Machine Learning (SVM)

In [31]:
import numpy as np
import pandas as pd
mnist = pd.read_csv('sample_data/mnist_train_small.csv', header=None) #헤더를 안주면 0인덱스줄이 헤더가 되버림!!! 
mnist.head(3)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,768,769,770,771,772,773,774,775,776,777,778,779,780,781,782,783,784
0,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [32]:
mnist.shape

(20000, 785)

In [33]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [34]:
X_train, X_test, y_train, y_test = train_test_split(
    mnist.iloc[:, 1:], mnist.iloc[:, 0], test_size=0.25, stratify=mnist.iloc[:,0], random_state=2021)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((15000, 784), (5000, 784), (15000,), (5000,))

In [35]:
#minmax scaler대신 나눠서
X_train = X_train.astype('float')/255
X_test = X_test.astype('float')/255

In [36]:
# 머신러닝은 GPU쓰나안쓰나 속도비슷; 하이퍼파라미터안만지고 그냥 모델생성했음;
sv_clf = SVC()
sv_clf.fit(X_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [37]:
pred = sv_clf.predict(X_test)
accuracy_score(y_test, pred)

0.9714

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
#판다스니까 values로 넘파이로 바꾼다음에 reshape(28,28)
digit = X_test.iloc[1000,:].values.reshape(28,28)
plt.figure(figsize=(4,4))
plt.xticks([]); plt.yticks([])
plt.imshow(digit, cmap=plt.cm.binary, interpolation='nearest')
plt.show()

#### 2. Deep Learning

In [39]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [40]:
# One-hot Encoding 사이킷런말고 케라스에서원핫하는방법
from tensorflow.keras.utils import to_categorical
Y_train = to_categorical(y_train)
Y_test = to_categorical(y_test)
Y_train.shape, Y_test.shape

((15000, 10), (5000, 10))

In [42]:
# Model 생성
model = Sequential()
model.add(Dense(512, input_shape=(784,), activation='relu')) #인풋:784개의 데이터를 받아서 512개로 풀리커넥티드네크웍이만들어짐;....
model.add(Dense(100, activation='relu'))
model.add(Dense(10, activation='softmax')) #또는 시그모이드  아웃풋10개
model.summary() #784 * W512 + b512의 갯수 param 구하는게 딥러닝의 결과를 만들어낸다!? 단순계산반복 이럴때 GPU쓰면 속도빠름!

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dense_1 (Dense)              (None, 100)               51300     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1010      
Total params: 454,230
Trainable params: 454,230
Non-trainable params: 0
_________________________________________________________________


In [43]:
# 모델의 실행환경설정(3가지 설정)
# (경사하강법)loss가 작아지도록 조절하는게 학습!
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) 
#다중분류할때categorical_crossentrop 이진분류는banery 회기는 rmse 루트민스퀘어/평가지표metrics

In [44]:
# 모델 실행
model.fit(X_train, Y_train, validation_split=0.2, epochs=20, batch_size=200) # batch_size 15000개중 200개씩 가지고 들어와서 훈련 너무크면 메모리가커야함;속도는빠른데, 갱신주기가 길어져 결과안좋을수있다
#결과값 loss accuracy val_loss val_accuracy : val_loss가 낮은게 가장 좋은 모델

Epoch 1/20
60/60 [==============================] - 2s 22ms/step - loss: 1.0158 - accuracy: 0.7032 - val_loss: 0.2809 - val_accuracy: 0.9160
Epoch 2/20
60/60 [==============================] - 1s 16ms/step - loss: 0.2481 - accuracy: 0.9315 - val_loss: 0.2134 - val_accuracy: 0.9360
Epoch 3/20
60/60 [==============================] - 1s 16ms/step - loss: 0.1617 - accuracy: 0.9558 - val_loss: 0.1839 - val_accuracy: 0.9493
Epoch 4/20
60/60 [==============================] - 1s 16ms/step - loss: 0.1083 - accuracy: 0.9715 - val_loss: 0.1734 - val_accuracy: 0.9500
Epoch 5/20
60/60 [==============================] - 1s 15ms/step - loss: 0.0790 - accuracy: 0.9795 - val_loss: 0.1661 - val_accuracy: 0.9513
Epoch 6/20
60/60 [==============================] - 1s 15ms/step - loss: 0.0661 - accuracy: 0.9835 - val_loss: 0.1598 - val_accuracy: 0.9567
Epoch 7/20
60/60 [==============================] - 1s 17ms/step - loss: 0.0431 - accuracy: 0.9909 - val_loss: 0.1517 - val_accuracy: 0.9563
Epoch 8/20
60

In [45]:
model.evaluate(X_test, Y_test)
#앞숫자loss0.117 /뒤숫자accuracy 0.9692

157/157 [==============================] - 1s 4ms/step - loss: 0.1171 - accuracy: 0.9692


[0.11712192744016647, 0.9692000150680542]

In [46]:
acc = model.evaluate(X_test, Y_test)[1]
print(f'Accuracy = {acc:.4f}') #0.9692

157/157 [==============================] - 1s 4ms/step - loss: 0.1171 - accuracy: 0.9692
Accuracy = 0.9692
